# Project Milestone 4: Preliminary Results

## Imports 

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

## Feature extraction 

Load the data.

In [36]:
data = pd.read_csv('mbti_1.csv')
data = data.replace({c: i for i, c in enumerate(pd.unique(data['type']))})
data.head()

,type,posts
0,0,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,1,'I'm finding the lack of me in these posts ver...
2,2,'Good one _____ https://www.youtube.com/wat...
3,3,"'Dear INTP, I enjoyed our conversation the o..."
4,4,'You're fired.|||That's another silly misconce...


Create feature matrix of raw word frequencies.

In [48]:
vectorizer = CountVectorizer(input='content')
X = vectorizer.fit_transform(list(data['posts']))
y = data['type'].values

print('n_classes =', X.shape[0])
print('n_features =', X.shape[1])

n_classes = 8675
n_features = 145412


Split into testing and training.

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Training samples: {}'.format(len(y_train)))
print('Testing samples: {}'.format(len(y_test)))

Training samples: 6940
Testing samples: 1735


### Feature selection
See https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection. For example, we could use `mutual_info_classif` with `SelectKBest` to keep the $K$ best features according to their mutual information score. Or could do something else for now like PCA.

## Classification

Random guessing.

In [32]:
y_pred = np.random.randint(0, 16, size=y_test.shape)
print('Accuracy = {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy = 0.06


Support vector machine (linear kernel).

In [33]:
clf = svm.LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


## Performance evalutation
I suppose we should use $k$-fold cross validation.

In [34]:
print('Accuracy = {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy = 0.60
